In [ ]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
import numpy as np
from tqdm import tqdm

def load_model(model_name, df):
    df['corpus_vector'] = ''
    # corpus_vector = pd.DataFrame([columns=])
    model= Doc2Vec.load(model_name)
    for i in range(len(df)):
        df['corpus_vector'].iloc[i] = model.infer_vector(df['Content'].iloc[i].split())
    return df

def dbscan(df):
    """Function to form dbscan clusters and display them"""
#     eps = 0.005# how close points should be to each other to be considered a part of a cluster 
#     min_samples = 3# the minimum number of points to form a dense region  
#     dbscan = DBSCAN( eps=eps, min_samples=min_samples,metric = "cosine" ) 
#     dbscan_model = dbscan.fit(corpus_vector)
    
    pca = PCA(n_components=2)
    result = pca.fit_transform(df['corpus_vector'].to_list())
    print(result.shape)
    db = DBSCAN(eps=0.002, min_samples=3)
    dbscan_model = db.fit(result)
    #Forming the clusters

    core_samples_mask = np.zeros_like(dbscan_model.labels_, dtype=bool)
    core_samples_mask[dbscan_model.core_sample_indices_] = True
    labels1 = dbscan_model.labels_
    print(labels1)
    n_clusters_ = len(set(labels1)) - (1 if -1 in labels1 else 0) # Number of clusters in labels
    # print(labels1)
    print(len(labels1))
    # print(n_clusters_) # number of clusters
    df['ClusterNum'] =''
    clusters1 = {} # a dictionary for different cluster 
    for c, i in enumerate(labels1):
        if i == -1:
            continue
        elif i in clusters1:
            clusters1[i].append(df['ID'].iloc[c])
            df['ClusterNum'].iloc[c] = i
        else:
            clusters1[i] = [df['ID'].iloc[c]]
            
            df['ClusterNum'].iloc[c] = i
    with open('./cluster.txt', 'w') as f:
        for c in clusters1:
            f.write("Cluster No."+" "+str(c)+" "+str(clusters1[c]))
            f.write('--------------\n')
            f.write('\n')  
    # for c in clusters1: # print the different clusters
    #     print("Cluster No."+" "+str(c)+" "+str(clusters1[c]))
    #     print()
    
    return df, clusters1

In [ ]:

df = pd.read_csv('/Users/stillssi/Desktop/NEWS_TEAM_4/MDL/content_based/sample_df.csv')
df = load_model("/Users/stillssi/Desktop/NEWS_TEAM_4/MDL/content_based/model/trainedMDL",df)


In [ ]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df, cluster1 = dbscan(df)

In [ ]:
df

In [ ]:
df.drop(['CosSimilarity'], axis=1, inplace=True)
df_1 = df[df['ClusterNum'] != '']
cluster_df = pd.DataFrame(data = [cluster1])
cluster_df=cluster_df.transpose()

In [ ]:
cluster_df.to_csv('./cluster_list.csv')

In [ ]:
cluster_df = pd.read_csv('/Users/stillssi/Desktop/NEWS_TEAM_4/MDL/content_based/cluster_list.csv')

In [ ]:
cluster_df.columns

In [ ]:
cluster_df.rename(columns={'Unnamed: 0':'cluster'}, inplace=True)
cluster_df.rename(columns={'0':'list'}, inplace=True)

In [ ]:
cluster_df

In [ ]:
cluster_df.set_index('cluster')

In [ ]:
df = df[df['ClusterNum'] != '']

In [ ]:
#df에서 ClusterNum 칼럼 값 가져옴 -> cluster_df칼럼 (df벡터와 cluster_df의 리스트의 뉴스ID의 벡터와 cos계산)

In [ ]:
len(cluster1)

In [ ]:
df.head(3)

In [ ]:
df = df.set_index('ID')

In [ ]:
df

In [ ]:
df['cossim_list'] = ''

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
df.loc[886057]


In [ ]:
from tqdm import tqdm
import itertools
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(df)):
    cosine_sim_list = []
    # 뉴스 아이디 리스트
    id_list = cluster_df.loc[cluster_df['cluster'] == df['ClusterNum'].iloc[i], 'list']
    tmp = id_list.values[0].replace('[','')
    tmp = tmp.replace(']','')
    tmp = tmp.split(',')
    
    id_list = list(map(int, tmp))
    # print(id_list)
    id_list_df = df.loc[id_list]
    # print(id_list_df)
    vec1 = df['corpus_vector'].iloc[i]
    vec1 = np.reshape(vec1,(100,1))

    id_list_df = id_list_df['corpus_vector'].apply(lambda x: np.reshape(x, (100,1)))
    for j in id_list_df.index:
        cosine_result = cos_sim(vec1.T, id_list_df[j])
        cosine_result = cosine_result.tolist()
        if cosine_result[0][0] == 1.0:
            pass
        else:
            cosine_sim_list.append((j,cosine_result[0][0]))
    
    df['cossim_list'].iloc[i] = cosine_sim_list


In [ ]:
for i in df['cossim_list']:
    i.sort(key=lambda x: (x[0], -x[1]), reverse=True)

In [ ]:
df.to_csv('./content_based_df.csv')

In [ ]:
content_df = df.copy()

In [ ]:
content_df['sim_rank'] = ''

In [ ]:
k = 0
for i in content_df['cossim_list']:
    sim_list = []
    for j in range(len(i)):
        sim_list.append(i[j][0])
    content_df['sim_rank'].iloc[k] = sim_list
    k +=1

In [ ]:
content_df.drop(['corpus_vector', 'ClusterNum','cossim_list'], axis=1, inplace=True)

In [ ]:
content_df.rename(columns={"sim_rank": "cossimilarity"}, inplace=True)

In [ ]:
import traceback
import pymysql
class Mysql:

    def __init__(self, db_config:dict) -> None:
        '''
        생성자 메서드
        인스턴스 생성 시 db_config를 전달받아 DB에 연결합니다.
        '''
        self.DB = pymysql.connect(**db_config)
        return

    def insert_many(self, table:str, columns: str, values: list) -> bool:
            """
            Insert Many Datas
            예시)
            table = "Students"
            columns = "name, email, phone"
            values = [
                ('이름', '이메일', '번호'),
                ...
            ]
            """
            

            sql = f"INSERT INTO {table}({columns}) " \
                    "VALUES ("  + ','.join(["%s"]*len(values[0])) + ");"
            try:
                
                with self.DB.cursor() as cur:
                    cur.executemany(sql, values)
                    self.DB.commit()
                return True
            except:
                traceback.print_exc()
                return False

In [ ]:
content_df['cossimilarity'] = content_df['cossimilarity'].apply(lambda x: ','.join(s for s in x))

In [ ]:
content_df['cossimilarity'] = content_df['cossimilarity'].apply(lambda x: x.replace(']',''))
content_df['cossimilarity']=content_df['cossimilarity'].apply(lambda x: x.replace('[',''))

In [135]:
df_con = {'host':'localhost','port':3306,'user':'root','password':'12345678','db':'news_rec','charset':'utf8'}
db_insert = Mysql(df_con)

In [136]:
table = 'tb_news_team_4'
col = "DomainID, MainCategory, SubCategory, WritedAt, Title,Content, URL, PhotoURL, Writer, Stickers, Press,cossimilarity"

In [144]:
content_df = content_df.fillna('')

In [145]:
da = []
for i in range(len(content_df)):
    data_list = []
    data = content_df.iloc[i].values
    for i in range(len(data)):
        data_list.append(data[i])
    data = tuple(data_list)
    da.append(data)



In [146]:
db_insert.insert_many(table=table, columns=col, values=da)

True

In [149]:
import pymysql
import pandas as pd
host_name = 'localhost'
host_port = 3306
username='root'
password='12345678'
database_name='news_rec'
db = pymysql.connect(
    host=host_name,
    port=host_port,
    user=username,
    password=password,
    db=database_name,
    charset='utf8'
)
SQL='SELECT * FROM tb_news_team_4'
df=pd.read_sql(SQL, db)

/var/folders/z9/0vcwpql57g75_wt83hs0h89h0000gn/T/ipykernel_29640/3219018108.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(SQL, db)


In [150]:
df

,ID,DomainID,MainCategory,SubCategory,WritedAt,Title,Content,URL,PhotoURL,Writer,Stickers,Press,CosSimilarity
0,1,2,스포츠,농구,2022-11-27 16:11:00,"아반도, 날카로운 골밑 돌파! [포토]",돌 실내 경기도 서울 열린 파고다 경기 안양 아반 체육관 수비,https://v.daum.net/v/YoIX22A092,https://t1.daumcdn.net/news/202211/27/sportsdo...,김민성 기자,"{'추천해요': 0, '좋아요': 0, '감동이에요': 0, '화나요': 0, '슬...",스포츠동아,"1249668, 949955, 944830, 868607, 84458"
1,2,2,스포츠,축구,2022-11-14 09:39:00,"""김민재, 호날두-수아레스와 일대일 가능... 손흥민 최고"" ESPN",같 월드컵 아시아 한국 수아레스 이어 김민재 선보일 보유고 소개 능력 피지컬 선수 ...,https://v.daum.net/v/1czH3hZdrj,https://t1.daumcdn.net/news/202211/14/poctan/2...,우충원,"{'추천해요': 12, '좋아요': 161, '감동이에요': 14, '화나요': 4...",OSEN,"889553, 874244, 72094, 54353"
2,3,2,스포츠,야구,2022-11-25 19:59:00,박동원 LG행으로 사라진 18홈런…누가 KIA의 대포 갈증을 해결할까,시즌 다황 이룰 대인 주 필요 생산력 효상 임석진 위한 선박 보강 동원 금지 재재 ...,https://v.daum.net/v/7hF6gGfO8x,https://t1.daumcdn.net/news/202211/25/mydaily/...,마이데일리,"{'추천해요': 1, '좋아요': 20, '감동이에요': 1, '화나요': 81, ...",마이데일리,"907574, 888463, 86906"
3,4,1,스포츠,야구,2022-11-30 17:01:00,"파노니도 바꿀 수 있다 140km 2점대 ERA 좌좌→150km 우우, KIA 외인...",타이거즈 우완 다 구상 염두 나온다면 바꿀 수다 기자 같 제공 투수 교체 펀치 원투...,https://sports.news.naver.com/news?oid=109&aid...,['https://imgnews.pstatic.net/image/109/2022/1...,이선호,"{'좋아요': 147, '슬퍼요': 3, '화나요': 15, '팬이에요': 10, ...",OSEN,"969100, 964532, 937377, 931381, 923472, 917990..."
4,5,2,스포츠,야구,2022-11-20 19:06:00,김유성 '국민의례 마치고 박수'[포토],오후 몬스터즈 다 송파구 잠실 서울 박수 연습 베어스 열린 잠실야구장 최강 경기 김...,https://v.daum.net/v/tjGDTHujoH,https://t1.daumcdn.net/news/202211/20/xportsne...,김한준 기자,"{'추천해요': 0, '좋아요': 0, '감동이에요': 0, '화나요': 6, '슬...",엑스포츠뉴스,"1299012, 1286520, 976357, 954321, 948450, 9276..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13910,13911,2,연예,해외연예,2022-11-15 00:00:00,"다니엘 크레이그, “제임스 본드로 돌아가지 않겠다”[해외이슈]",아웃 나아갈 이브스 크레이그 어니언 위한 돌아온다 한편 단전 금지 영화 배포 글래스...,https://v.daum.net/v/20221118094356821,https://img1.daumcdn.net/thumb/R658x0.q70/?fna...,입력,,마이데일리,"1334309, 1334286, 1333959, 1333276, 1332530, 1..."
13911,13912,2,연예,연예일반,2022-11-15 00:00:00,"아이리스(IRRIS), 눈 뗄 수 없는 매력 ‘Stay W!th Me’ 퍼포먼스 비디오",성숙 리브 채널 반응 전망 끌 기자 에너지 유튜브 공개 글로벌 비디오 시 니나 싱글...,https://v.daum.net/v/20221125220043377,https://img3.daumcdn.net/thumb/R658x0.q70/?fna...,손봉석,,스포츠경향,"970805, 954107"
13912,13913,2,연예,연예일반,2022-11-15 00:00:00,"'세븐♥' 이다해, 뷰티 MC의 패션은 시상식이네…화려함 그 잡채",다 스타일링 이다해 기자 완성 세븐 공개 매치 셔츠 애 가수 드레스,https://v.daum.net/v/20221127161102212,https://img2.daumcdn.net/thumb/R658x0.q70/?fna...,강민경,,텐아시아,"1259186, 887183, 19873"
13913,13914,2,연예,방송,2022-11-15 00:00:00,"'편스토랑' 박수홍, 20년 인연 지인에 오열 ""모두가 등 돌릴 때 남아줬어"" [T...",인연 지혜 기자 눈물 수연 이날 사람 다 전보 박수홍,https://v.daum.net/v/20221202220209324,https://img2.daumcdn.net/thumb/R658x0.q70/?fna...,임시령,,스포츠투데이,"1333042, 1327554, 1323032, 1304638, 1302469, 1..."
